In [48]:
import requests, json
from typing import List, Dict
import hashlib
from datetime import datetime, timedelta

APP_KEY="uvw0th4nksi97o1bgqp35zjxr6e2may8",
APP_SECRET="9450cbbbb22be056537e82138f1fa15220656e9b",
ACCESS_TOKEN="9949a4aea79acad2e22e501e89c5ff3146f15e48",
REST_ID="1c8w7fp500",
PETPOOJA_COOKIE="mgnhpm6a8r5u11gatkpqhg7q00"

In [49]:
PETPOOJA_URL = "https://api.petpooja.com/V1/thirdparty/generic_get_orders/"
WEBHOOK_URL = "https://webhooks.db1-prod-dachnona.store/webhooks/petpooja/orders/"
#WEBHOOK_URL = "http://localhost:8000/webhooks/petpooja/orders/" #

def hash_order(order: dict) -> str:
    """
    Create a stable hash for an order dict.
    Ensures consistent ordering.
    """
    canonical = json.dumps(order, sort_keys=True, separators=(",", ":"))
    return hashlib.sha256(canonical.encode("utf-8")).hexdigest()

def date_range(start_date: str, end_date: str) -> list[str]:
    """
    Returns a list of dates between start_date and end_date (inclusive)
    in 'YYYY-MM-DD' format.

    :param start_date: 'YYYY-MM-DD'
    :param end_date: 'YYYY-MM-DD'
    """
    start = datetime.strptime(start_date, "%Y-%m-%d").date()
    end = datetime.strptime(end_date, "%Y-%m-%d").date()

    if start > end:
        return []

    return [
        (start + timedelta(days=i)).strftime("%Y-%m-%d")
        for i in range((end - start).days + 1)
    ]

def transform_petpooja_orders(order_json):
    transformed = []

    for order in order_json:
        # ------------------ TAX ------------------
        taxes = []
        for tax in order.get("Tax", []):
            taxes.append({
                "rate": float(tax.get("rate", 0)),
                "type": tax.get("type", ""),
                "title": tax.get("title", ""),
                "amount": float(tax.get("amount", 0))
            })

        # ------------------ ORDER ------------------
        o = order.get("Order", {})
        order_block = {
            "total": float(o.get("total", 0)),
            "biller": o.get("custom_payment_type", "") or o.get("order_from", ""),
            "status": o.get("status", ""),
            "comment": o.get("comment", ""),
            "orderID": int(o.get("orderID", 0)),
            "assignee": "",
            "table_no": o.get("table_no", ""),
            "token_no": "",
            "round_off": o.get("round_off", "0"),
            "tax_total": float(o.get("tax_total", 0)),
            "core_total": float(o.get("core_total", 0)),
            "created_on": o.get("created_on", ""),
            "order_from": o.get("order_from", ""),
            "order_type": o.get("order_type", ""),
            "payment_type": o.get("payment_type", ""),
            "no_of_persons": int(o.get("no_of_persons", 0)),
            "order_from_id": o.get("order_from_id", ""),
            "discount_total": float(o.get("discount_total", 0)),
            "service_charge": o.get("service_charge", 0),
            "sub_order_type": o.get("sub_order_type", ""),
            "delivery_charges": float(o.get("delivery_charges", 0)),
            "packaging_charge": float(o.get("container_charges", 0)),
            "customer_invoice_id": o.get("orderID", "")
        }

        # ------------------ CUSTOMER ------------------
        c = order.get("Customer", {})
        customer_block = {
            "name": c.get("name", ""),
            "gstin": c.get("gst_no", ""),
            "phone": c.get("phone", ""),
            "address": c.get("address", "")
        }

        # ------------------ DISCOUNT ------------------
        discounts = []
        for d in order.get("Discount", []):
            discounts.append({
                "rate": float(d.get("rate", 0)),
                "type": d.get("type", ""),
                "title": d.get("title", ""),
                "amount": float(d.get("amount", 0))
            })

        # ------------------ ITEMS ------------------
        items = []
        for item in order.get("OrderItem", []):
            items.append({
                "tax": float(item.get("total_tax", 0)),
                "name": item.get("name", ""),
                "addon": item.get("addon", []),
                "price": float(item.get("price", 0)),
                "total": float(item.get("total", 0)),
                "itemid": int(item.get("itemid", 0)),
                "discount": float(item.get("total_discount", 0)),
                "itemcode": item.get("itemcode", ""),
                "quantity": int(item.get("quantity", 1)),
                "sap_code": item.get("itemsapcode", ""),
                "specialnotes": item.get("specialnotes", ""),
                "category_name": item.get("categoryname", ""),
                "vendoritemcode": ""
            })

        # ------------------ RESTAURANT ------------------
        r = order.get("Restaurant", {})
        restaurant_block = {
            "restID": r.get("restID", ""),
            "address": r.get("address", ""),
            "res_name": r.get("res_name", ""),
            "contact_information": r.get("contact_information", "")
        }

        # ------------------ FINAL PAYLOAD ------------------
        transformed.append({
            "event": "orderdetails",
            "token": "",
            "properties": {
                "Tax": taxes,
                "Order": order_block,
                "Customer": customer_block,
                "Discount": discounts,
                "OrderItem": items,
                "Restaurant": restaurant_block
            }
        })

    return transformed

def fetch_petpooja_orders_simple(date_str):
    url = "https://api.petpooja.com/V1/thirdparty/generic_get_orders/"

    payload = {
        "app_key": APP_KEY,
        "app_secret": APP_SECRET,
        "access_token": ACCESS_TOKEN,
        "restID": REST_ID,
        "order_date": date_str,
        "refId": ""
    }

    headers = {
        "Content-Type": "application/json",
        "Cookie": "PETPOOJA_API="+PETPOOJA_COOKIE
    }

    response = requests.post(
        url,
        data=json.dumps(payload),  # EXACT match to curl --data
        headers=headers
    )

    return response.json()

def get_orders_for_date_range(date_list):
    all_orders = []
    seen_hashes = set()

    for date_str in date_list:
        response = fetch_petpooja_orders_simple(date_str)
        orders = response.get("order_json", [])

        for order in orders:
            order_hash = hash_order(order)

            if order_hash in seen_hashes:
                continue  # exact duplicate

            seen_hashes.add(order_hash)
            all_orders.append(order)
    
    return all_orders


def push_orders_to_dach_nona(orders):
    transformed_orders = transform_petpooja_orders(orders)

    for payload in transformed_orders:
        order_id = payload["properties"]["Order"].get("orderID")

        if not order_id:
            print("⚠️ Skipping order with missing orderID")
            continue

        headers = {
            "Content-Type": "application/json",
            "X-Idempotency-Key": f"petpooja:order:{order_id}"
        }

        response = requests.post(
            WEBHOOK_URL,
            json=payload,
            headers=headers,
            timeout=20
        )

        if response.status_code not in (200, 201, 202):
            print("❌ Failed", order_id, response.status_code, response.text)

def sync_petpooja_orders_for_date_range(start_date: str, end_date: str):
    dates = date_range(start_date, end_date)
    combined_orders = get_orders_for_date_range(dates)
    push_orders_to_dach_nona(combined_orders)
    

In [50]:
dates = date_range("2026-01-04", "2026-01-04")
combined_orders = get_orders_for_date_range(dates)

In [63]:
push_orders_to_dach_nona(combined_orders)

In [26]:
import json
import requests
import pandas as pd
from typing import Optional, Iterable


BASE_URL = "http://localhost:8000/analytics"
API_KEY = "f3e1753aa4c44159fa7218a31cd8db1e"

HEADERS = {
    "X-API-Key": API_KEY,
}

JSON_COLUMNS = {
    "orders": ["raw_event"],
    "order-items": ["raw_item"],
    "addons": ["raw_addon"],
    "discounts": ["raw_discount"],
}

def _safe_json_load(value):
    if value is None:
        return None
    if isinstance(value, (dict, list)):
        return value
    try:
        return json.loads(value)
    except (TypeError, ValueError):
        return value

def fetch_stream(
    endpoint: str,
    limit: int = 500,
    start_cursor: Optional[int] = 0,
) -> pd.DataFrame:
    rows = []
    last_stream_id = start_cursor or 0

    while True:
        params = {
            "limit": limit,
            "cursor": last_stream_id,   # ✅ correct param
        }

        resp = requests.get(
            f"{BASE_URL}/{endpoint}/",
            headers=HEADERS,
            params=params,
            timeout=60,
        )
        resp.raise_for_status()

        payload = resp.json()
        batch = payload.get("data", [])
        if not batch:
            break

        rows.extend(batch)

        # advance cursor using the last row actually returned
        last_stream_id = batch[-1]["stream_id"]

        # ✅ CRITICAL: stop when fewer than limit rows are returned
        if not batch:
            break

    df = pd.DataFrame(rows)

    for col in JSON_COLUMNS.get(endpoint, []):
        if col in df.columns:
            df[col] = df[col].apply(_safe_json_load)

    return df


def fetch_petpooja_data(
    limit: int = 500,
    start_cursor: Optional[int] = 0
) -> pd.DataFrame:
    """
    Fetch orders stream and safely parse JSON columns.
    """
    return fetch_stream(
        endpoint="orders",
        limit=limit,
        start_cursor=start_cursor
    )

def _extract_order_fields(e):
    if not isinstance(e, dict):
        return None, None
    order = (
        e.get("raw_payload", {})
         .get("properties", {})
         .get("Order", {})
    )
    return order.get("orderID"), order.get("created_on")


def dedupe_latest_orders(df: pd.DataFrame) -> pd.DataFrame:
    """
    Keep only the latest snapshot per order_id.
    Assumes each payload is a full replacement.
    """

    if "raw_event" not in df.columns:
        raise ValueError("Expected 'raw_event' column")

    out = df.copy()

    # ---- Extract once ----
    out[["_order_id", "_created_on"]] = out["raw_event"].apply(
        lambda e: pd.Series(_extract_order_fields(e))
    )

    # ---- Clean ----
    out["_created_on"] = pd.to_datetime(out["_created_on"], errors="coerce")
    out = out[out["_order_id"].notna()]

    # ---- Deduplicate ----
    out = (
        out
        .sort_values("_created_on")
        .drop_duplicates(subset=["_order_id"], keep="last")
        .drop(columns=["_order_id", "_created_on"])
    )

    return out

def normalize_orders(orders_df: pd.DataFrame) -> dict[str, pd.DataFrame]:
    """
    Normalize raw order events into structured DataFrames.
    """

    if "raw_event" not in orders_df.columns:
        raise ValueError("orders_df must contain a 'raw_event' column")

    events = orders_df["raw_event"].dropna()
    restaurants = {}
    orders, customers = [], []
    order_items, addons = [], []
    discounts, taxes = [], []

    customer_id = 1
    order_item_id = 1
    addon_id = 1
    discount_id = 1
    tax_id = 1

    for event in events:
        payload = event.get("raw_payload", {})
        props = payload.get("properties", {})
        if not props:
            continue

        # ---------- Restaurant ----------
        r = props.get("Restaurant", {})
        restID = r.get("restID")
        if restID and restID not in restaurants:
            restaurants[restID] = {
                "restID": restID,
                "address": r.get("address"),
                "res_name": r.get("res_name"),
                "contact_information": r.get("contact_information")
            }

        # ---------- Order ----------
        o = props.get("Order", {})
        order_id = o.get("orderID")
        if not order_id:
            continue

        # Convert created_on ONCE
        order_created_on = pd.to_datetime(
            o.get("created_on"), errors="coerce"
        )

        # ---------- Customer ----------
        c = props.get("Customer", {})
        customers.append({
            "customer_id": customer_id,
            "order_id": order_id,
            "order_created_on": order_created_on,
            "name": c.get("name"),
            "gstin": c.get("gstin"),
            "phone": c.get("phone"),
            "address": c.get("address")
        })

        orders.append({
            "order_id": order_id,
            "restID": restID,
            "customer_id": customer_id,
            **o,
            "created_on": order_created_on
        })
        customer_id += 1

        # ---------- Order Items ----------
        for item in props.get("OrderItem", []):
            order_items.append({
                "order_item_id": order_item_id,
                "order_id": order_id,
                "order_created_on": order_created_on,
                "itemid": item.get("itemid"),
                "itemcode": item.get("itemcode"),
                "name": item.get("name"),
                "category_name": item.get("category_name"),
                "quantity": item.get("quantity"),
                "price": item.get("price"),
                "discount": item.get("discount"),
                "tax": item.get("tax"),
                "total": item.get("total"),
                "specialnotes": item.get("specialnotes"),
                "vendoritemcode": item.get("vendoritemcode")
            })

            for a in item.get("addon", []):
                addons.append({
                    "order_item_addon_id": addon_id,
                    "order_item_id": order_item_id,
                    "order_id": order_id,
                    "order_created_on": order_created_on,
                    "addon_id": a.get("addon_id"),
                    "name": a.get("name"),
                    "group_name": a.get("group_name"),
                    "addon_group_id": a.get("addon_group_id"),
                    "price": a.get("price"),
                    "quantity": a.get("quantity"),
                    "sap_code": a.get("sap_code")
                })
                addon_id += 1

            order_item_id += 1

        # ---------- Discounts ----------
        for d in props.get("Discount", []):
            discounts.append({
                "discount_id": discount_id,
                "order_id": order_id,
                "order_created_on": order_created_on,
                "rate": d.get("rate"),
                "type": d.get("type"),
                "title": d.get("title"),
                "amount": d.get("amount")
            })
            discount_id += 1

        # ---------- Taxes ----------
        for t in props.get("Tax", []):
            taxes.append({
                "tax_id": tax_id,
                "order_id": order_id,
                "order_created_on": order_created_on,
                "rate": t.get("rate"),
                "type": t.get("type"),
                "title": t.get("title"),
                "amount": t.get("amount")
            })
            tax_id += 1

    return {
        "restaurants": pd.DataFrame(restaurants.values()),
        "orders": pd.DataFrame(orders),
        "customers": pd.DataFrame(customers),
        "order_items": pd.DataFrame(order_items),
        "addons": pd.DataFrame(addons),
        "discounts": pd.DataFrame(discounts),
        "taxes": pd.DataFrame(taxes),
    }

def prepare_orders_data():
    orders_df = fetch_petpooja_data()
    return normalize_orders(dedupe_latest_orders(orders_df))

def add_order_date(df_orders: pd.DataFrame) -> pd.DataFrame:
    df = df_orders.copy()
    #df["created_on"] = pd.to_datetime(df["created_on"], errors="coerce")
    df = df[df["created_on"].notna()]
    df["order_date"] = df["created_on"].dt.date
    return df

def daily_aggregate(
    df: pd.DataFrame,
    metrics: dict,
    group_cols: list[str] = ["order_date"]
) -> pd.DataFrame:
    """
    Generic daily aggregation helper.
    """
    return (
        df
        .groupby(group_cols, as_index=False)
        .agg(**metrics)
        .sort_values(group_cols)
    )

def daily_channel_revenue(df_orders: pd.DataFrame) -> pd.DataFrame:
    return (
        df_orders
        .groupby(["order_date", "order_from"])
        .agg(revenue=("total", "sum"))
        .reset_index()
        .pivot(index="order_date", columns="order_from", values="revenue")
        .fillna(0)
        .reset_index()
    )

def daily_item_sales(order_items_df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean item names and compute daily item-level sales metrics.

    Returns:
        DataFrame with:
        - order_date
        - item_name
        - items_sold
        - total_sales
    """
    df = order_items_df.copy()

    # ---- Validate required columns ----
    required_cols = {"name", "quantity", "total", "order_created_on"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # ---- Clean item names ----
    df["item_name"] = (
        df["name"]
        .astype(str)
        .str.strip()
        .str.lower()
    )

    # ---- Ensure datetime ----
    df["order_created_on"] = pd.to_datetime(
        df["order_created_on"], errors="coerce"
    )

    df = df[df["order_created_on"].notna()]

    # ---- Extract date ----
    df["order_date"] = df["order_created_on"].dt.date

    # ---- Daily aggregation ----
    result = (
        df
        .groupby(["order_date", "item_name"], as_index=False)
        .agg(
            items_sold=("quantity", "sum"),
            total_sales=("total", "sum"),
        )
        .sort_values(["order_date", "items_sold"], ascending=[True, False])
    )

    return result


In [27]:
tables = prepare_orders_data()
df_orders = add_order_date(tables["orders"])

In [28]:
daily_base = daily_aggregate(
    df_orders,
    metrics={
        "total_revenue": ("total", "sum"),
        "net_revenue": ("core_total", "sum"),
        "tax_collected": ("tax_total", "sum"),
        "total_orders": ("order_id", "nunique"),
    }
)
daily_channel = daily_channel_revenue(df_orders)
daily_revenue_combined = (
    daily_base
    .merge(daily_channel, on="order_date", how="left")
)

/var/folders/q5/z0f360hx7kv29538mfdmy7s00000gn/T/ipykernel_26319/261321578.py:300: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_orders


In [32]:
daily_revenue_combined[:-2]

,order_date,total_revenue,net_revenue,tax_collected,total_orders,Home Website,POS,Swiggy,Zomato
0,2025-10-31,19589.0,18470.0,932.68,25,1526.0,8170.0,1307.0,8586.0
1,2025-11-01,37843.0,36337.5,1801.8,52,0.0,14223.0,3014.0,20606.0
2,2025-11-02,30514.0,29645.0,1452.66,49,0.0,9252.0,6045.0,15217.0
3,2025-11-03,17954.0,17510.0,854.88,26,0.0,4532.0,1050.0,12372.0
4,2025-11-04,25995.0,25635.0,1237.62,39,0.0,6144.0,5481.0,14370.0
5,2025-11-05,24969.0,24455.0,1188.9,40,0.0,7550.0,3009.0,14410.0
6,2025-11-06,16055.0,15860.0,764.36,28,0.0,6658.0,662.0,8735.0
7,2025-11-07,26949.0,26585.0,1283.2,36,0.0,4584.0,9987.0,12378.0
8,2025-11-08,31663.0,30845.0,1502.88,39,0.0,16035.0,2731.0,12897.0
9,2025-11-09,27575.0,26825.0,1313.02,36,0.0,12736.0,4111.0,10728.0


In [71]:
len(combined_orders)

63

In [64]:
i = []
for x in combined_orders:
    if '2026-01-03' in x['Order']['created_on']:
        i.append(x['Order']['orderID'])

In [70]:
set(i)

{'5308',
 '5309',
 '5310',
 '5311',
 '5312',
 '5313',
 '5314',
 '5315',
 '5316',
 '5317',
 '5318',
 '5319',
 '5320',
 '5321',
 '5322',
 '5323',
 '5324',
 '5325',
 '5326',
 '5327',
 '5328',
 '5329',
 '5330',
 '5331',
 '5332',
 '5333',
 '5334',
 '5335',
 '5336'}

In [5]:
from datetime import datetime
from decimal import Decimal
from collections import defaultdict

payloads = combined_orders

latest_orders = {}

# Step 1: Deduplicate by orderID (keep latest)
for payload in payloads:
    order = payload.get("Order", {})
    order_id = order.get("orderID")
    created_on = order.get("created_on")

    if not order_id or not created_on:
        continue

    created_dt = datetime.strptime(created_on, "%Y-%m-%d %H:%M:%S")

    if order_id not in latest_orders:
        latest_orders[order_id] = (created_dt, payload)
    else:
        existing_dt = latest_orders[order_id][0]
        if created_dt > existing_dt:
            latest_orders[order_id] = (created_dt, payload)

# Step 2: Aggregate revenue per day
revenue_per_day = defaultdict(Decimal)

for _, payload in latest_orders.values():
    order = payload["Order"]
    order_date = order["created_on"][:10]  # YYYY-MM-DD
    total = Decimal(str(order.get("total", "0")))
    revenue_per_day[order_date] += total

# Step 3: Print results (sorted by date)
for day in sorted(revenue_per_day):
    print(day, "->", revenue_per_day[day])


2026-01-03 -> 19746
2026-01-04 -> 1008


In [70]:
from datetime import datetime

TARGET_DATE = "2025-12-26"
latest_orders = {}

# Step 1: keep latest payload per orderID
for payload in payloads:
    order = payload.get("Order", {})
    order_id = order.get("orderID")
    created_on = order.get("created_on")

    if not order_id or not created_on:
        continue

    created_dt = datetime.strptime(created_on, "%Y-%m-%d %H:%M:%S")

    if order_id not in latest_orders or created_dt > latest_orders[order_id][0]:
        latest_orders[order_id] = (created_dt, payload)

# Step 2: collect order IDs for the target date
order_ids_2025_12_26 = [
    order_id
    for order_id, (_, payload) in latest_orders.items()
    if payload["Order"]["created_on"].startswith(TARGET_DATE)
]

print(order_ids_2025_12_26)
print("Total orders:", len(order_ids_2025_12_26))


['5077', '5076', '5075', '5074', '5073', '5072', '5071', '5070', '5069', '5068', '5067', '5066', '5065', '5064', '5063', '5062', '5061', '5060', '5059', '5058', '5057', '5056', '5055']
Total orders: 23


In [73]:
payloads[0]['Order']['orderID']

'4196'

In [76]:
y = None
for x in payloads:
    if x['Order']['orderID'] == '5055':
        y = x

In [77]:
y

{'Restaurant': {'restaurantid': '353446',
  'res_name': 'Dach & Nona',
  'address': 'House 2173, Ramgarh Dhani, Sector 67, Gurgaon\n Gurugram Haryana 122102',
  'restaurant_state': 'Haryana',
  'contact_information': '7428846234',
  'restID': '1c8w7fp500'},
 'Customer': {'name': 'Payal Mehta',
  'address': ' Delhi NCR India',
  'phone': '',
  'gst_no': '',
  'created_date': ''},
 'Order': {'orderID': '5055',
  'refId': '7408459389',
  'delivery_charges': '0',
  'container_charges': '20',
  'order_type': 'Delivery',
  'sub_order_type_id': '1161857',
  'sub_order_type': 'Zomato',
  'payment_type': 'Online',
  'custom_payment_type': 'Zomato',
  'table_no': '',
  'no_of_persons': '0',
  'discount_total': '100',
  'tax_total': '42.5',
  'round_off': '0.50',
  'core_total': '930',
  'total': '893',
  'created_on': '2025-12-26 12:28:00',
  'order_from': 'Zomato',
  'advance_order': 'No',
  'part_payment': [],
  'order_date': '2025-12-26',
  'status': 'Success',
  'service_charge': 0,
  'waive

In [78]:
y

{'Restaurant': {'restaurantid': '353446',
  'res_name': 'Dach & Nona',
  'address': 'House 2173, Ramgarh Dhani, Sector 67, Gurgaon\n Gurugram Haryana 122102',
  'restaurant_state': 'Haryana',
  'contact_information': '7428846234',
  'restID': '1c8w7fp500'},
 'Customer': {'name': 'Payal Mehta',
  'address': ' Delhi NCR India',
  'phone': '',
  'gst_no': '',
  'created_date': ''},
 'Order': {'orderID': '5055',
  'refId': '7408459389',
  'delivery_charges': '0',
  'container_charges': '20',
  'order_type': 'Delivery',
  'sub_order_type_id': '1161857',
  'sub_order_type': 'Zomato',
  'payment_type': 'Online',
  'custom_payment_type': 'Zomato',
  'table_no': '',
  'no_of_persons': '0',
  'discount_total': '100',
  'tax_total': '42.5',
  'round_off': '0.50',
  'core_total': '930',
  'total': '893',
  'created_on': '2025-12-26 12:28:00',
  'order_from': 'Zomato',
  'advance_order': 'No',
  'part_payment': [],
  'order_date': '2025-12-26',
  'status': 'Success',
  'service_charge': 0,
  'waive

In [26]:
transformed_orders = transform_petpooja_orders([combined_orders[1]])

In [31]:
combined_orders[-1]

{'Restaurant': {'restaurantid': '353446',
  'res_name': 'Dach & Nona',
  'address': 'House 2173, Ramgarh Dhani, Sector 67, Gurgaon\n Gurugram Haryana 122102',
  'restaurant_state': 'Haryana',
  'contact_information': '7428846234',
  'restID': '1c8w7fp500'},
 'Customer': {'name': 'shreyashi sharma',
  'address': ' Sector 65, Gurgaon Delhi NCR India, Pincode:0',
  'phone': '',
  'gst_no': '',
  'created_date': ''},
 'Order': {'orderID': '5337',
  'refId': '7480299437',
  'delivery_charges': '0',
  'container_charges': '20',
  'order_type': 'Delivery',
  'sub_order_type_id': '1161857',
  'sub_order_type': 'Zomato',
  'payment_type': 'Online',
  'custom_payment_type': 'Zomato',
  'table_no': '',
  'no_of_persons': '0',
  'discount_total': '100',
  'tax_total': '14',
  'round_off': '0',
  'core_total': '360',
  'total': '294',
  'created_on': '2026-01-04 14:10:46',
  'order_from': 'Zomato',
  'advance_order': 'No',
  'part_payment': [],
  'order_date': '2026-01-04',
  'status': 'Success',
 

In [32]:
combined_orders[-1]

{'Restaurant': {'restaurantid': '353446',
  'res_name': 'Dach & Nona',
  'address': 'House 2173, Ramgarh Dhani, Sector 67, Gurgaon\n Gurugram Haryana 122102',
  'restaurant_state': 'Haryana',
  'contact_information': '7428846234',
  'restID': '1c8w7fp500'},
 'Customer': {'name': 'shreyashi sharma',
  'address': ' Sector 65, Gurgaon Delhi NCR India, Pincode:0',
  'phone': '',
  'gst_no': '',
  'created_date': ''},
 'Order': {'orderID': '5337',
  'refId': '7480299437',
  'delivery_charges': '0',
  'container_charges': '20',
  'order_type': 'Delivery',
  'sub_order_type_id': '1161857',
  'sub_order_type': 'Zomato',
  'payment_type': 'Online',
  'custom_payment_type': 'Zomato',
  'table_no': '',
  'no_of_persons': '0',
  'discount_total': '100',
  'tax_total': '14',
  'round_off': '0',
  'core_total': '360',
  'total': '294',
  'created_on': '2026-01-04 14:10:46',
  'order_from': 'Zomato',
  'advance_order': 'No',
  'part_payment': [],
  'order_date': '2026-01-04',
  'status': 'Success',
 

In [46]:
x = []
for y in combined_orders:
    dt = y['Order']['created_on']
    if '2026-01-04' in dt:
        x.append(y)

In [47]:
len(x)

34